In [1]:
import win32com.client as win32
import pandas as pd
import logging
import os
import tkinter as tk
from tkinter import ttk, filedialog

# 로그 설정 (디버깅 수준으로 설정)
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# 한글 객체 초기화
hwp = win32.gencache.EnsureDispatch("hwpframe.hwpobject")
hwp.RegisterModule("FilePathCheckDLL", "FilePathCheckerModuleExample")
hwp.XHwpWindows.Item(0).Visible = True  # 한글 창을 보이게 설정

# 엑셀 파일 경로 및 시트 설정 함수
def 엑셀열기(file_path_excel, sheet_name="참여자 명단"):
    excel = pd.read_excel(file_path_excel, sheet_name=sheet_name, header=1, dtype=str)
    excel_data = excel.fillna("")
    return excel_data

# 파일 열기 함수
def 파일열기(hwp, file_name, file_path):
    file_path = fr'{file_path}\{file_name}.hwp'
    return hwp.Open(file_path)

# Tkinter GUI 구성
class DocumentAutomationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("한글 서식 자동화 도구")
        self.root.geometry("500x400")
        
        # 엑셀 파일 경로 변수와 라벨
        self.excel_file_path = tk.StringVar()
        self.excel_label = tk.Label(root, text="엑셀 파일 경로:")
        self.excel_label.pack(pady=5)
        self.excel_entry = tk.Entry(root, textvariable=self.excel_file_path, width=50)
        self.excel_entry.pack(pady=5)
        
        # 엑셀 파일 선택 버튼
        self.excel_button = tk.Button(root, text="엑셀 파일 선택", command=self.select_excel_file)
        self.excel_button.pack(pady=5)
        
        # 서식 파일 경로 변수와 라벨
        self.form_file_path = tk.StringVar()
        self.form_label = tk.Label(root, text="서식 파일 경로:")
        self.form_label.pack(pady=5)
        self.form_entry = tk.Entry(root, textvariable=self.form_file_path, width=50)
        self.form_entry.pack(pady=5)
        
        # 서식 파일 선택 버튼
        self.form_button = tk.Button(root, text="서식 파일 선택", command=self.select_form_file)
        self.form_button.pack(pady=5)
        
        # 드롭다운 메뉴로 서식 선택
        self.form_types = ["[서식33] 인턴형 일경험 참여신청서", "[서식46] 인턴형 참여청년 확인서"]
        self.form_selection = ttk.Combobox(root, values=self.form_types)
        self.form_selection.set("서식을 선택하세요")
        self.form_selection.pack(pady=10)
        
        # 실행 버튼
        self.execute_button = tk.Button(root, text="실행", command=self.execute)
        self.execute_button.pack(pady=20)

    def select_excel_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("Excel files", "*.xlsx;*.xls")])
        self.excel_file_path.set(file_path)

    def select_form_file(self):
        file_path = filedialog.askopenfilename(filetypes=[("HWP files", "*.hwp")])
        self.form_file_path.set(file_path)

    def execute(self):
        excel_path = self.excel_file_path.get()
        form_path = self.form_file_path.get()
        selected_form = self.form_selection.get()

        if not excel_path or not form_path or selected_form == "서식을 선택하세요":
            print("모든 필드를 올바르게 선택하세요.")
            return

        if selected_form == "[서식33] 인턴형 일경험 참여신청서":
            self.process_form_33(excel_path, form_path)
        elif selected_form == "[서식46] 인턴형 참여청년 확인서":
            self.process_form_46(excel_path, form_path)
        else:
            print("지원하지 않는 서식입니다.")

    def process_form_33(self, excel_path, form_path):
        for index, row in 엑셀열기(excel_path).iterrows():
            파일열기(hwp, "[서식33] 인턴형 일경험 참여신청서", os.path.dirname(form_path))

            if not row['성명']:  # 성명 값이 없는 경우 현재 행을 건너뜀
                print(f"빈 값 발견 - 행 {index + 1} 건너뜀")
                continue

            try:
                hwp.PutFieldText('성명', row['성명'])
                hwp.PutFieldText('주민등록번호', row['주민등록번호'])
                hwp.PutFieldText('만나이', row['만나이'])
                hwp.PutFieldText('주소', row['주소'])
                hwp.PutFieldText('연락처', row['연락처'])
                hwp.PutFieldText('이메일', row['이메일'])
                hwp.PutFieldText('최종학과', row['최종학과'])
                hwp.PutFieldText('작성날짜', row['작성날짜'])
                hwp.PutFieldText('최종학교명', row['최종학교명'])
                
                if row['최종학력구분'] == '대학교_4년제':
                    hwp.PutFieldText('대학교', '■')
                    hwp.PutFieldText('대학교명', row['최종학교명'])
                    if row['최종학력상태'] == '졸업':
                        hwp.PutFieldText('대학교졸업', '■')
                    elif row['최종학력상태'] == '졸업예정자':
                        hwp.PutFieldText('졸업예정자', '■')
                    elif row['최종학력상태'] == '휴학':
                        if row['학년'] in ['1','2']:
                            hwp.PutFieldText('재학(1~2학년)', '■')
                        elif row['학년'] in ['3','4']:
                            hwp.PutFieldText('재학(3~4학년)', '■')
                    elif row['최종학력상태'] == '재학':
                        if row['학년'] in ['1','2']:
                            hwp.PutFieldText('재학(1~2학년)', '■')
                        elif row['학년'] in ['3','4']:
                            hwp.PutFieldText('재학(3~4학년)', '■')
                elif row['최종학력구분'] == '고등학교':
                    hwp.PutFieldText('고등학교', '■')
                    hwp.PutFieldText('고등학교명', row['최종학교명'])
                    if row['최종학력상태'] == '졸업':
                        hwp.PutFieldText('고등학교졸업', '■')
                    elif row['최종학력상태'] == '졸업예정자':
                        hwp.PutFieldText('고등학교졸업예정자', '■')
                    elif row['최종학력상태'] == '재학':
                        if row['학년'] in ['1', '2']:
                            hwp.PutFieldText('고등학교재학(1~2학년)', '■')
                        elif row['학년'] == '3':
                            hwp.PutFieldText('고등학교재학(3학년)', '■')
                    elif row['최종학력상태'] == '검정고시':
                        hwp.PutFieldText('검정고시', '■')
                
                new_file_name = f'[서식33] 인턴형 일경험 참여신청서_{row["성명"]}.hwp'
                new_file_path = os.path.join(os.path.dirname(form_path), new_file_name)
                hwp.SaveAs(new_file_path)
                print(f'파일 저장 완료: {new_file_name}')
                hwp.Clear(1)
            except Exception as e:
                print(f"오류 발생 (행 {index + 1}): {e}")

    def process_form_46(self, excel_path, form_path):
        for index, row in 엑셀열기(excel_path).iterrows():
            파일열기(hwp, "[서식46] 인턴형 참여청년 확인서", os.path.dirname(form_path))

            if not row['성명']:  # 성명 값이 없는 경우 현재 행을 건너뜀
                print(f"빈 값 발견 - 행 {index + 1} 건너뜀")
                continue

            try:
                hwp.PutFieldText('성명', row['성명'])
                hwp.PutFieldText('생년월일', row['생년월일'])
                hwp.PutFieldText('작성날짜', row['작성날짜'])
                
                new_file_name = f'[서식46] 인턴형 참여청년 확인서_{row["성명"]}.hwp'
                new_file_path = os.path.join(os.path.dirname(form_path), new_file_name)
                hwp.SaveAs(new_file_path)
                print(f'파일 저장 완료: {new_file_name}')
                hwp.Clear(1)
            except Exception as e:
                print(f"오류 발생 (행 {index + 1}): {e}")

# Tkinter GUI 실행
if __name__ == "__main__":
    root = tk.Tk()
    app = DocumentAutomationApp(root)
    root.mainloop()

# 엑셀 반복이 종료되면 한글 종료
hwp.Quit()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Python310\lib\site-packages\pandas\core\indexes\base.py", line 3805, in get_loc
    return self._engine.get_loc(casted_key)
  File "index.pyx", line 167, in pandas._libs.index.IndexEngine.get_loc
  File "index.pyx", line 196, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7081, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\\_libs\\hashtable_class_helper.pxi", line 7089, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: '성명'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "c:\Python310\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\thdus\AppData\Local\Temp\ipykernel_31368\2766104200.py", line 84, in execute
    self.process_form_33(excel_path, form_path)
  File "C:\Users\thdus\AppData\Local\Temp\ipykernel_31368\27661